In [8]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [9]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [10]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster','snow_m',
                          'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56', 'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']
    else:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster',
                          'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56', 'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+str(VER)+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]

In [11]:
def make_lag_roll(LAG_DAY,lag_df_new):
   
    lag_df = base_test[['id','d',TARGET]]
    
    lag_df=lag_df.sort_values(by=["d"])
  
    for i in range(0,len(LAG_DAY)):

        shift_day = LAG_DAY[i][0]
        roll_wind = LAG_DAY[i][1]
        col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
        lag_df[col_name] = (lag_df.groupby(['id'])[TARGET]).transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    lag_df_new=lag_df.drop(columns=["sales"])
    return lag_df_new

In [12]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    "lambda":0.1,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 



# lgb_params ={
#         "objective" : "tweedie",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
#         "metric": ["rmse"],
#         "nthread": -1,
#         "tweedie_variance_power":1.1,
#     'verbosity': 1,
# #     'num_iterations' : 1500,
#     'num_leaves': 128,
#     "min_data_in_leaf": 104,
#     }




# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [13]:
VER = 5                        # Our model version
SEED = 42                      # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1941               # End day of our train set, change this part for final
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28_v3.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_evaluation.csv')['store_id']#change this part for final
STORES_IDS = list(STORES_IDS.unique())

#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        ROLS_SPLIT.append([i,j])

In [14]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    print(features_columns)
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                            label=grid_df[train_mask][TARGET],
                            weight=grid_df[train_mask]['sell_price'])
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                            label=grid_df[valid_mask][TARGET],
                            weight=grid_df[valid_mask]['sell_price'])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+str(VER)+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    imp_type = "gain"
    features = estimator.feature_name()
    importances = estimator.feature_importance(imp_type)
    importance_df=pd.DataFrame(features,columns=['features'])
    importance_df['importances']=importances
    importance_df=importance_df.sort_values(by='importances', ascending=False)
    importance_df.to_csv(store_id+'_fe_imp_'+str(VER)+'.csv',index=False)
    del importance_df
    gc.collect()
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    #     !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()

Train CA_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7',

[100]	valid_0's rmse: 1.19552
[200]	valid_0's rmse: 1.17112
[300]	valid_0's rmse: 1.16137
[400]	valid_0's rmse: 1.15512
[500]	valid_0's rmse: 1.15038
[600]	valid_0's rmse: 1.14603
[700]	valid_0's rmse: 1.1422
[800]	valid_0's rmse: 1.13849
[900]	valid_0's rmse: 1.13498
[1000]	valid_0's rmse: 1.13143
[1100]	valid_0's rmse: 1.12836
[1200]	valid_0's rmse: 1.12549
[1300]	valid_0's rmse: 1.12253
[1400]	valid_0's rmse: 1.11954
Train TX_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm

[100]	valid_0's rmse: 1.29663
[200]	valid_0's rmse: 1.26327
[300]	valid_0's rmse: 1.25419
[400]	valid_0's rmse: 1.24713
[500]	valid_0's rmse: 1.2416
[600]	valid_0's rmse: 1.23668
[700]	valid_0's rmse: 1.23153
[800]	valid_0's rmse: 1.22736
[900]	valid_0's rmse: 1.22324
[1000]	valid_0's rmse: 1.22012
[1100]	valid_0's rmse: 1.21632
[1200]	valid_0's rmse: 1.21334
[1300]	valid_0's rmse: 1.21016
[1400]	valid_0's rmse: 1.20734


In [15]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day


for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    
    
    lag_df_new = pd.DataFrame()

    lag_df_new=make_lag_roll(ROLS_SPLIT,lag_df_new)


    grid_df = grid_df.merge(lag_df_new, on=['id','d'], how='left')


    for store_id in STORES_IDS:
        
        if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
                              'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept',
                              'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 
                              'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 
                              'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 
                              'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend',
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d',
                              'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 
                              'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 
                              'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36',
                              'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 
                              'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 
                              'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 
                              'rolling_mean_168', 'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14',
                              'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 'rolling_mean_tmp_7_7', 
                              'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56']
        else:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
                              'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept',
                              'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 
                              'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m',
                              'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 
                              'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend',
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d',
                              'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 
                              'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 
                              'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36',
                              'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 
                              'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 
                              'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 
                              'rolling_mean_168', 'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14',
                              'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 'rolling_mean_tmp_7_7', 
                              'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56']
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id

        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()

    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    
    del temp_df, lag_df_new

all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  4.70 min round |  4.70 min total |  40022.45 day sales |
Predict | Day: 2
##########  4.69 min round |  9.38 min total |  37571.07 day sales |
Predict | Day: 3
##########  4.66 min round |  14.04 min total |  36975.99 day sales |
Predict | Day: 4
##########  4.63 min round |  18.67 min total |  37245.93 day sales |
Predict | Day: 5
##########  4.64 min round |  23.31 min total |  42771.22 day sales |
Predict | Day: 6
##########  4.66 min round |  27.97 min total |  50664.98 day sales |
Predict | Day: 7
##########  4.63 min round |  32.60 min total |  51184.89 day sales |
Predict | Day: 8
##########  4.65 min round |  37.25 min total |  45971.93 day sales |
Predict | Day: 9
##########  4.66 min round |  41.90 min total |  38775.92 day sales |
Predict | Day: 10
##########  4.63 min round |  46.53 min total |  43869.28 day sales |
Predict | Day: 11
##########  4.63 min round |  51.16 min total |  45208.85 day sales |
Predict | Day: 12
##########  4.74 min roun

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.870053,0.772245,0.769220,0.795715,1.146603,1.270989,1.171651,1.105411,0.884802,...,0.923505,1.318675,1.220026,0.955526,0.905121,0.866556,0.927998,1.132739,1.235258,1.118773
1,HOBBIES_1_002_CA_1_evaluation,0.227708,0.186179,0.186104,0.186404,0.210585,0.270438,0.296708,0.218849,0.187559,...,0.277902,0.307211,0.321109,0.195927,0.209423,0.202146,0.223316,0.276085,0.337831,0.325235
2,HOBBIES_1_003_CA_1_evaluation,0.553375,0.487176,0.534532,0.533247,0.744177,0.874407,0.816062,0.527403,0.463380,...,0.567604,0.730777,0.735459,0.433800,0.394575,0.422795,0.492775,0.668737,0.762911,0.750434
3,HOBBIES_1_004_CA_1_evaluation,1.524919,1.343165,1.266637,1.205075,1.837776,2.824238,3.163879,2.051375,1.390310,...,1.817656,2.681844,3.175846,1.564844,1.272872,1.191450,1.242909,1.818223,2.638524,2.562997
4,HOBBIES_1_005_CA_1_evaluation,1.103604,0.957963,0.921227,0.903564,1.078556,1.460549,1.408799,1.208367,0.903357,...,1.209900,1.524049,1.578020,1.027695,0.976358,0.971569,1.039537,1.420345,1.688591,1.420405


In [16]:
all_preds.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_evaluation,0.566869,0.500762,0.477404,0.510216,0.515191,0.561108,0.619037,0.552469,0.471204,...,0.560149,0.709725,0.799307,0.521672,0.541387,0.598276,0.507819,0.525563,0.587537,0.682566
30486,FOODS_3_824_WI_3_evaluation,0.283228,0.253773,0.248052,0.242228,0.238393,0.284519,0.324373,0.273995,0.257407,...,0.283187,0.392423,0.450147,0.306145,0.337094,0.320820,0.281785,0.260043,0.282924,0.329259
30487,FOODS_3_825_WI_3_evaluation,0.596229,0.484860,0.437998,0.448241,0.516697,0.587079,0.684791,0.680018,0.463650,...,0.777157,1.196752,1.287175,0.988607,1.037454,0.976770,0.714914,0.724589,0.701344,0.779672
30488,FOODS_3_826_WI_3_evaluation,0.971171,1.062714,0.916046,0.925824,1.016534,1.169492,1.133873,1.293228,1.007259,...,1.176715,1.411353,1.417329,1.120050,1.328460,1.300708,1.152698,1.195560,1.247312,1.204441
30489,FOODS_3_827_WI_3_evaluation,1.859251,1.639658,1.498519,1.638778,1.980240,2.079078,1.825210,1.902430,1.671518,...,1.552772,1.870983,1.947113,1.557956,1.807889,1.691179,1.587526,1.735612,1.748167,1.792215


In [17]:
all_preds.shape

(30490, 29)

In [18]:
all_preds.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.312642,1.232243,1.212725,1.221579,1.402795,1.661692,1.678744,1.507771,1.271759,1.438809,...,1.529237,1.916189,1.937766,1.502465,1.406196,1.479292,1.373575,1.542821,1.779633,1.643072
std,2.692904,2.521170,2.464363,2.503607,2.982485,3.476375,3.460230,3.110851,2.524221,2.839684,...,3.248288,4.056144,4.077053,3.052064,2.958006,3.114497,2.875216,3.340522,3.895156,3.601144
min,0.001596,0.014109,0.014867,0.015593,0.017073,0.017710,0.016881,0.013529,0.013443,0.018285,...,0.019001,0.019434,0.017953,0.011726,0.012954,0.011934,0.014488,0.016590,0.017459,0.015388
25%,0.263450,0.270897,0.267686,0.269197,0.299817,0.349621,0.352529,0.315112,0.281297,0.314419,...,0.317411,0.389711,0.392618,0.306759,0.284794,0.300563,0.292689,0.324215,0.365333,0.332320
50%,0.604603,0.583156,0.569781,0.572111,0.645756,0.769066,0.785500,0.697525,0.603032,0.677703,...,0.687030,0.862898,0.882677,0.686880,0.634374,0.662553,0.626129,0.697286,0.797968,0.749583
75%,1.323992,1.228574,1.216645,1.217070,1.386796,1.650819,1.681081,1.483683,1.274376,1.428191,...,1.491946,1.863059,1.903529,1.497874,1.369169,1.439810,1.332563,1.492537,1.717541,1.600766
max,105.520643,98.511159,97.531474,101.360791,139.015821,158.615236,149.097379,121.513990,97.095690,100.080292,...,143.387498,171.462447,176.900226,115.576060,103.349454,102.386727,106.237946,137.927385,169.169447,153.677042


In [19]:
# all the following is changed

In [20]:
# replace validation part
train_df = pd.read_csv('sales_train_evaluation.csv')
train_df=train_df[['id','d_1914','d_1915','d_1916','d_1917','d_1918','d_1919','d_1920','d_1921','d_1922','d_1923',
                  'd_1924','d_1925','d_1926','d_1927','d_1928','d_1929','d_1930','d_1931','d_1932','d_1933',
                  'd_1934','d_1935','d_1936','d_1937','d_1938','d_1939','d_1940','d_1941']]

In [21]:
train_df.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [22]:
submission = pd.read_csv('sample_submission.csv')

In [23]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
submission.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
train_df['id']=train_df['id'].str.replace('evaluation','validation')

In [26]:
train_df.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [27]:
train_df.columns=submission.columns

In [28]:
train_df.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [29]:
train_df.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_validation,0,0,0,2,2,0,0,0,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_validation,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_validation,0,0,1,1,0,2,1,1,0,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_validation,1,3,0,1,2,1,0,2,1,...,1,1,1,4,6,0,1,1,1,0
30489,FOODS_3_827_WI_3_validation,0,0,0,0,0,1,1,1,2,...,1,2,0,5,4,0,2,2,5,1


In [30]:
train_df.shape

(30490, 29)

In [31]:
submission.shape

(60980, 29)

In [32]:
submission = submission[['id']]
sub1 = submission.merge(train_df, on=['id'], how='left')

In [33]:
sub1.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [34]:
sub1.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60976,FOODS_3_824_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60977,FOODS_3_825_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60978,FOODS_3_826_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60979,FOODS_3_827_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
sub1=sub1[:30490]

In [36]:
sub1.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [37]:
sub1.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_validation,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,...,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
30486,FOODS_3_824_WI_3_validation,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
30487,FOODS_3_825_WI_3_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0
30488,FOODS_3_826_WI_3_validation,1.0,3.0,0.0,1.0,2.0,1.0,0.0,2.0,1.0,...,1.0,1.0,1.0,4.0,6.0,0.0,1.0,1.0,1.0,0.0
30489,FOODS_3_827_WI_3_validation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,...,1.0,2.0,0.0,5.0,4.0,0.0,2.0,2.0,5.0,1.0


In [38]:
sub2 = submission.merge(all_preds, on=['id'], how='left')

In [39]:
sub2.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
sub2.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.566869,0.500762,0.477404,0.510216,0.515191,0.561108,0.619037,0.552469,0.471204,...,0.560149,0.709725,0.799307,0.521672,0.541387,0.598276,0.507819,0.525563,0.587537,0.682566
60976,FOODS_3_824_WI_3_evaluation,0.283228,0.253773,0.248052,0.242228,0.238393,0.284519,0.324373,0.273995,0.257407,...,0.283187,0.392423,0.450147,0.306145,0.337094,0.320820,0.281785,0.260043,0.282924,0.329259
60977,FOODS_3_825_WI_3_evaluation,0.596229,0.484860,0.437998,0.448241,0.516697,0.587079,0.684791,0.680018,0.463650,...,0.777157,1.196752,1.287175,0.988607,1.037454,0.976770,0.714914,0.724589,0.701344,0.779672
60978,FOODS_3_826_WI_3_evaluation,0.971171,1.062714,0.916046,0.925824,1.016534,1.169492,1.133873,1.293228,1.007259,...,1.176715,1.411353,1.417329,1.120050,1.328460,1.300708,1.152698,1.195560,1.247312,1.204441
60979,FOODS_3_827_WI_3_evaluation,1.859251,1.639658,1.498519,1.638778,1.980240,2.079078,1.825210,1.902430,1.671518,...,1.552772,1.870983,1.947113,1.557956,1.807889,1.691179,1.587526,1.735612,1.748167,1.792215


In [41]:
sub2=sub2[30490:]

In [42]:
sub2.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0.870053,0.772245,0.769220,0.795715,1.146603,1.270989,1.171651,1.105411,0.884802,...,0.923505,1.318675,1.220026,0.955526,0.905121,0.866556,0.927998,1.132739,1.235258,1.118773
30491,HOBBIES_1_002_CA_1_evaluation,0.227708,0.186179,0.186104,0.186404,0.210585,0.270438,0.296708,0.218849,0.187559,...,0.277902,0.307211,0.321109,0.195927,0.209423,0.202146,0.223316,0.276085,0.337831,0.325235
30492,HOBBIES_1_003_CA_1_evaluation,0.553375,0.487176,0.534532,0.533247,0.744177,0.874407,0.816062,0.527403,0.463380,...,0.567604,0.730777,0.735459,0.433800,0.394575,0.422795,0.492775,0.668737,0.762911,0.750434
30493,HOBBIES_1_004_CA_1_evaluation,1.524919,1.343165,1.266637,1.205075,1.837776,2.824238,3.163879,2.051375,1.390310,...,1.817656,2.681844,3.175846,1.564844,1.272872,1.191450,1.242909,1.818223,2.638524,2.562997
30494,HOBBIES_1_005_CA_1_evaluation,1.103604,0.957963,0.921227,0.903564,1.078556,1.460549,1.408799,1.208367,0.903357,...,1.209900,1.524049,1.578020,1.027695,0.976358,0.971569,1.039537,1.420345,1.688591,1.420405


In [43]:
sub2.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.566869,0.500762,0.477404,0.510216,0.515191,0.561108,0.619037,0.552469,0.471204,...,0.560149,0.709725,0.799307,0.521672,0.541387,0.598276,0.507819,0.525563,0.587537,0.682566
60976,FOODS_3_824_WI_3_evaluation,0.283228,0.253773,0.248052,0.242228,0.238393,0.284519,0.324373,0.273995,0.257407,...,0.283187,0.392423,0.450147,0.306145,0.337094,0.320820,0.281785,0.260043,0.282924,0.329259
60977,FOODS_3_825_WI_3_evaluation,0.596229,0.484860,0.437998,0.448241,0.516697,0.587079,0.684791,0.680018,0.463650,...,0.777157,1.196752,1.287175,0.988607,1.037454,0.976770,0.714914,0.724589,0.701344,0.779672
60978,FOODS_3_826_WI_3_evaluation,0.971171,1.062714,0.916046,0.925824,1.016534,1.169492,1.133873,1.293228,1.007259,...,1.176715,1.411353,1.417329,1.120050,1.328460,1.300708,1.152698,1.195560,1.247312,1.204441
60979,FOODS_3_827_WI_3_evaluation,1.859251,1.639658,1.498519,1.638778,1.980240,2.079078,1.825210,1.902430,1.671518,...,1.552772,1.870983,1.947113,1.557956,1.807889,1.691179,1.587526,1.735612,1.748167,1.792215


In [44]:
final_sub=pd.concat([sub1,sub2],axis=0)

In [45]:
final_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [46]:
final_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.566869,0.500762,0.477404,0.510216,0.515191,0.561108,0.619037,0.552469,0.471204,...,0.560149,0.709725,0.799307,0.521672,0.541387,0.598276,0.507819,0.525563,0.587537,0.682566
60976,FOODS_3_824_WI_3_evaluation,0.283228,0.253773,0.248052,0.242228,0.238393,0.284519,0.324373,0.273995,0.257407,...,0.283187,0.392423,0.450147,0.306145,0.337094,0.320820,0.281785,0.260043,0.282924,0.329259
60977,FOODS_3_825_WI_3_evaluation,0.596229,0.484860,0.437998,0.448241,0.516697,0.587079,0.684791,0.680018,0.463650,...,0.777157,1.196752,1.287175,0.988607,1.037454,0.976770,0.714914,0.724589,0.701344,0.779672
60978,FOODS_3_826_WI_3_evaluation,0.971171,1.062714,0.916046,0.925824,1.016534,1.169492,1.133873,1.293228,1.007259,...,1.176715,1.411353,1.417329,1.120050,1.328460,1.300708,1.152698,1.195560,1.247312,1.204441
60979,FOODS_3_827_WI_3_evaluation,1.859251,1.639658,1.498519,1.638778,1.980240,2.079078,1.825210,1.902430,1.671518,...,1.552772,1.870983,1.947113,1.557956,1.807889,1.691179,1.587526,1.735612,1.748167,1.792215


In [47]:
final_sub.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,...,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000
mean,1.292480,1.198066,1.171220,1.180353,1.404841,1.657822,1.709034,1.462003,1.362577,1.368814,...,1.487363,1.848681,1.951516,1.445919,1.338995,1.347976,1.292937,1.469213,1.734651,1.712615
std,2.977773,2.748499,2.690906,2.701284,3.306661,3.789242,3.746503,3.284097,3.127444,3.085738,...,3.458819,4.245890,4.402909,3.185833,3.043971,3.060749,2.916933,3.429283,3.993582,3.958100
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.059888,0.000000,0.000000,0.000000,...,0.000000,0.048998,0.098148,0.000000,0.000000,0.000000,0.000000,0.000000,0.027561,0.069780
50%,0.479059,0.446455,0.433432,0.433896,0.559689,0.749079,0.806040,0.608579,0.534407,0.555021,...,0.603911,0.874032,0.950817,0.604551,0.531263,0.537108,0.507836,0.613518,0.799485,0.782582
75%,1.247439,1.096864,1.066881,1.064510,1.405104,1.892628,2.000000,1.512830,1.288978,1.349918,...,1.514531,2.000000,2.000000,1.511429,1.299939,1.322786,1.222626,1.484260,2.000000,2.000000
max,133.000000,117.000000,113.000000,101.360791,196.000000,158.615236,149.097379,121.513990,143.000000,107.000000,...,143.387498,171.462447,187.000000,115.576060,103.349454,102.386727,106.237946,137.927385,169.169447,153.677042


In [48]:
final_sub.to_csv('lgb_bystore_final3.csv',index=False)